<font style='font-weight:750;font-size:16px;text-decoration:underline;'>Imports</font>

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import math
from random import sample 

<font style='font-weight:750;font-size:16px;text-decoration:underline;'>Functions</font>

In [2]:
def get_samples(data_list_, neg_mapping_):
    samples=[]
    for i in range(len(data_list_)):
        user_id , item_id = data_list_[i]
        neg_item_id=int(sample(neg_mapping_[user_id],1)[0])
        samples.append((user_id,item_id,neg_item_id))
    return samples

In [3]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [4]:
# performing BPR updates using SGD method
def bpr(data_list_, mapping_, neg_mapping_, U_, V_, learning_rate_):
    samples=get_samples(data_list_, neg_mapping_)
    for i in range(len(samples)):
        user_id, positive_movie_id, negative_movie_id =  samples[i]

        Xu = U_[user_id,:]
        Vi = V_[positive_movie_id,:]
        Vj = V_[negative_movie_id,:]
        XuT_Vi = Xu.T.dot(Vi)
        XuT_Vj = Xu.T.dot(Vj)

        U_[user_id,:] += learning_rate_*(1-sigmoid(XuT_Vi-XuT_Vj))*(Vi-Vj)
        V_[positive_movie_id,:] += learning_rate_*(1-sigmoid(XuT_Vi-XuT_Vj))*Xu
        V_[negative_movie_id,:] += learning_rate_*(1-sigmoid(XuT_Vi-XuT_Vj))*(-Xu)
    
    return U_, V_

In [5]:
# building ranking matrix 
def get_ranking(data_, U_, V_):
    movies_ranking_by_user = {}
    set_of_users = set(data_['UserID'].astype('int32'))
    set_of_items = set(data_['ItemID'].astype('int32'))
    #calculate the movies ranking for each user
    for user in set_of_users:
        movies_ranking_by_user[user] =[]
        for item in set_of_items:
            rank = U_[user,:].T.dot(V_[item,:])
            movies_ranking_by_user[user].append((item,rank))
        movies_ranking_by_user[user].sort(key=lambda x: x[1], reverse=True)
        
    #fill out the ranking matrix
    num_users = data_['UserID'].max()+1
    num_items = data_['ItemID'].max()+1
    ranking_matrix=np.zeros([2, 4])
    movies_ranking_by_user = {}
    movies_ranking_by_user[1] =  [500, 450, 300, 100]
    movies_ranking_by_user[2] =  [565, 400, 30, 1]
    for user_id in movies_ranking_by_user:
        for index, item in enumerate(movies_ranking_by_user[user_id]):
            item_id = item[0]
            ranking_matrix[user_id,item_id] = index+1
    
    return ranking_matrix, movies_ranking_by_user


# user1 [500, 450, 300, 100]
# user1 [(1,500), (2,450), (3,300), (4,100)]

In [35]:
ranking_matrix=np.zeros([3, 4])
movies_ranking_by_user = {}
movies_ranking_by_user[1] =  [(1,500), (2,450), (0,300), (3,100)]
movies_ranking_by_user[2] =  [(3,12000),(1,650), (2,352), (0,300) ]
for user_id in movies_ranking_by_user:
    for index, item in enumerate(movies_ranking_by_user[user_id]):
        item_id = item[0]
        ranking_matrix[user_id,item_id] = index

In [36]:
print(ranking_matrix)

[[0. 0. 0. 0.]
 [2. 0. 1. 3.]
 [3. 1. 2. 0.]]


In [6]:
def get_mpr(data_list_, ranking_):
    mpr = 0 
    for i in range(len(data_list_)):
        user_id , item_id = data_list_[i]
        rank = ranking_[user_id,item_id]
        mpr += rank/ranking_.shape[1]
    mpr /= len(data_list_)
    return mpr        

In [7]:
def get_precision_k(mapping_, ranking_, users_, k_):    
    tp=0
    fp=0
    precision_k=0
    for i in range(users_):
        for j in ranking_[i+1][0:k_]:
            if j[0] in mapping_[i+1]:
                tp+=1
            else: fp+=1
        
        precision_k+=(tp/(tp+fp))
        tp=0
        fp=0
    
    return precision_k/users_

In [8]:
# performing random search in order to find the best hyper-parameters
def evaluate(train_data_, train_data_list_, mapping_, neg_mapping_, val_mapping_, val_data_, val_data_list_):
    num_users = train_data_['UserID'].max()+1
    num_items = train_data_['ItemID'].max()+1
    best_model = {}
    lowest_mpr = 5
    evaluations = 1
    # in each evaluation we are genarting difrent set of hyper-parameters and train the model with them
    for eval in range(evaluations):
        print("*******************************************************************************************")
        print('evaluation #{}'.format(eval+1))
        
        K = int(np.random.uniform(5, 40))
        learning_rate=np.random.uniform(0.0001, 0.03)
        print("K: %d ; learning_rate = %.6f" % (K, learning_rate))
        U = np.random.normal(scale=1./K, size=(num_users, K))
        V = np.random.normal(scale=1./K, size=(num_items, K))
         
        training_process = []
        val_process = []
        iterations = 5
        lr=learning_rate
        for iter in range(iterations):   
            U, V = bpr(train_data_list_, mapping_, neg_mapping_, U, V, lr)
            lr*=0.9
            ranking, rank_per_user = get_ranking(train_data_, U, V)
            train_mpr = get_mpr(train_data_list_, ranking)
            val_mpr = get_mpr(val_data_list_, ranking)
            val_precision_k_1=get_precision_k(val_mapping_, rank_per_user, num_users-1, 1)
            val_precision_k_10=get_precision_k(val_mapping_, rank_per_user, num_users-1, 10)
            val_precision_k_50=get_precision_k(val_mapping_, rank_per_user, num_users-1, 50)
            training_process.append(train_mpr)
            val_process.append(val_mpr)
            print("Iteration: %d ; train_mpr = %.10f ; val_mpr = %.10f ; val_precision_k_1 = %.10f ; val_precision_k_10 = %.10f ; val_precision_k_50 = %.10f" % (iter+1, train_mpr, val_mpr, val_precision_k_1, val_precision_k_10, val_precision_k_50))
            # checking for early stopping
            if iter >1:
                if val_process[-1] > val_process[-2] and val_process[-1] > val_process[-3]:
                    # removing the last 2 elements from the list:
                    val_process = val_process[:len(val_process)-2]
                    break           
        # checking if we have found a better model 
        if val_process[-1] < lowest_mpr: 
            lowest_mpr = val_process[-1]
            best_model['K'] = K
            best_model['learning_rate'] = learning_rate
            best_model['val_mpr'] = val_mpr
            best_model['val_precision_k_1'] = val_precision_k_1
            best_model['val_precision_k_10'] = val_precision_k_10
            best_model['val_precision_k_50'] = val_precision_k_50
            print('The bast model is:\n', best_model)
    return best_model['K'], best_model['learning_rate']

<font style='font-weight:750;font-size:16px;text-decoration:underline;'>Execution</font>

In [9]:
#load the data,
data_real = pd.read_csv("Train.csv")
data = data_real#data_real[(data_real['UserID']<= 10000) & (data_real['UserID']<= 10000)]

random_test_real = pd.read_csv("RandomTest.csv")
random_test = random_test_real#random_test_real[(random_test_real['UserID']<= 10000) & (random_test_real['Item1']<= 10000) & (random_test_real['Item2']<= 10000)]

popularity_test_real = pd.read_csv("PopularityTest.csv")
popularity_test = popularity_test_real#popularity_test_real[(popularity_test_real['UserID']<= 10000) & (popularity_test_real['Item1']<= 10000) & (popularity_test_real['Item2']<= 10000)]


#split to train and val and make sure all the users in the val df too (using stratify=data['UserID'])
train_data, val_data = train_test_split(data, test_size=0.20, random_state=27, stratify=data['UserID'])

In [10]:
# list of train data
train_data_list = []
for i, record in train_data.iterrows():
    row = record
    user_id = int(row['UserID'])
    item_id = int(row['ItemID'])
    train_data_list.append((user_id,item_id))

In [11]:
# list of validation data
val_data_list = []
for i, record in val_data.iterrows():
    row = record
    user_id = int(row['UserID'])
    item_id = int(row['ItemID'])
    val_data_list.append((user_id,item_id))

In [12]:
# map for each user all the positive movies based on the train data
positive_movies_by_user = {}
for i, record in train_data.iterrows():
    row = record
    user_id = int(row['UserID'])
    item_id = int(row['ItemID'])
    if user_id not in positive_movies_by_user: positive_movies_by_user[user_id] = []
    positive_movies_by_user[user_id].append(item_id)

In [13]:
# map for each user all the negative movies based on the train data
users_count=train_data['UserID'].max()+1
items_count = train_data['ItemID'].max()+1
negative_movies_by_user = {}
for i in range(1,users_count):
    if i not in negative_movies_by_user: negative_movies_by_user[i] = []
    negative_movies_by_user[i]=list(set(range(1, items_count)) - set(positive_movies_by_user[i]))

In [14]:
# map for each user all the positive movies based on the validation data
val_positive_movies_by_user = {}
for i, record in val_data.iterrows():
    row = record
    user_id = int(row['UserID'])
    item_id = int(row['ItemID'])
    if user_id not in val_positive_movies_by_user: val_positive_movies_by_user[user_id] = []
    val_positive_movies_by_user[user_id].append(item_id)

In [15]:
best_K, best_learning_rate = evaluate(train_data, train_data_list, positive_movies_by_user, negative_movies_by_user, val_positive_movies_by_user, val_data, val_data_list) 

*******************************************************************************************
evaluation #1
K: 23 ; learning_rate = 0.012782
Iteration: 1 ; train_mpr = 0.4622568893 ; val_mpr = 0.4787010034 ; val_precision_k_1 = 0.0178807947 ; val_precision_k_10 = 0.0145033113 ; val_precision_k_50 = 0.0125231788
Iteration: 2 ; train_mpr = 0.2375875802 ; val_mpr = 0.2481743049 ; val_precision_k_1 = 0.0750000000 ; val_precision_k_10 = 0.0714735099 ; val_precision_k_50 = 0.0581622517
Iteration: 3 ; train_mpr = 0.1867061020 ; val_mpr = 0.1888131339 ; val_precision_k_1 = 0.0996688742 ; val_precision_k_10 = 0.0912251656 ; val_precision_k_50 = 0.0704801325
Iteration: 4 ; train_mpr = 0.1839982520 ; val_mpr = 0.1852843045 ; val_precision_k_1 = 0.1079470199 ; val_precision_k_10 = 0.0908112583 ; val_precision_k_50 = 0.0710960265
Iteration: 5 ; train_mpr = 0.1830932070 ; val_mpr = 0.1843080126 ; val_precision_k_1 = 0.1096026490 ; val_precision_k_10 = 0.0914735099 ; val_precision_k_50 = 0.0714172185
T

In [16]:
# map for each user all the positive movies based on the entire data
all_data_positive_movies_by_user = {}
for i, record in data.iterrows():
    row = record
    user_id = int(row['UserID'])
    item_id = int(row['ItemID'])
    if user_id not in all_data_positive_movies_by_user: all_data_positive_movies_by_user[user_id] = []
    all_data_positive_movies_by_user[user_id].append(item_id)

In [17]:
# map for each user all the negative movies based on the entire data
users_count=data['UserID'].max()+1
items_count = data['ItemID'].max()+1
all_data_negative_movies_by_user = {}
for i in range(1,users_count):
    if i not in all_data_negative_movies_by_user: all_data_negative_movies_by_user[i] = []
    all_data_negative_movies_by_user[i]=list(set(range(1, items_count)) - set(all_data_positive_movies_by_user[i]))

In [18]:
# list of all data
data_list = []
for i, record in data.iterrows():
    row = record
    user_id = int(row['UserID'])
    item_id = int(row['ItemID'])
    data_list.append((user_id,item_id))

In [19]:
# after the random search we have the final hyper-paramters
# now we want to train the model with the final hyper-paramters and predict for the test file
num_users = data['UserID'].max()+1
num_items = data['ItemID'].max()+1
final_U = np.random.randn(num_users, best_K)
final_V = np.random.randn(num_items, best_K)
process = []
ranking_list = []
iterations = 50
lr=best_learning_rate
for iter in range(iterations):
    final_U, final_V = bpr(data_list, all_data_positive_movies_by_user, all_data_negative_movies_by_user, final_U, final_V, lr)
    lr*=0.9
    ranking, rank_per_user = get_ranking(data, final_U, final_V)
    mpr = get_mpr(data_list, ranking)
    precision_k_1=get_precision_k(all_data_positive_movies_by_user, rank_per_user, num_users-1, 1)
    precision_k_10=get_precision_k(all_data_positive_movies_by_user, rank_per_user, num_users-1, 10)
    precision_k_50=get_precision_k(all_data_positive_movies_by_user, rank_per_user, num_users-1, 50)
    process.append(mpr)
    ranking_list.append(ranking)
    print("Iteration: %d ; mpr = %.10f ; precision_k_1 = %.10f ; precision_k_10 = %.10f ; precision_k_50 = %.10f" % (iter+1, mpr, precision_k_1, precision_k_10, precision_k_50))
    # checking for early stopping
    if iter >1:
        if process[-1] > process[-2] and process[-1] > process[-3]:
            # removing the last 2 elements from the lists:
            process = process[:len(process)-2]
            ranking_list = ranking_list[:len(ranking_list)-2]
            break           

final_ranking = ranking_list[-1]

Iteration: 1 ; mpr = 0.4439280383 ; precision_k_1 = 0.0240066225 ; precision_k_10 = 0.0271854305 ; precision_k_50 = 0.0318576159
Iteration: 2 ; mpr = 0.2851466050 ; precision_k_1 = 0.1218543046 ; precision_k_10 = 0.1362748344 ; precision_k_50 = 0.1433278146
Iteration: 3 ; mpr = 0.2284302529 ; precision_k_1 = 0.2250000000 ; precision_k_10 = 0.2404966887 ; precision_k_50 = 0.2253576159
Iteration: 4 ; mpr = 0.2061317660 ; precision_k_1 = 0.2971854305 ; precision_k_10 = 0.2989238411 ; precision_k_50 = 0.2661920530
Iteration: 5 ; mpr = 0.1933342950 ; precision_k_1 = 0.3433774834 ; precision_k_10 = 0.3364900662 ; precision_k_50 = 0.2915960265


In [20]:
# filling out the random test file accroding to the predicted ranking
random_test['bitClassification'] = None
for i, record in random_test.iterrows():
    row = record
    user_id = int(row['UserID'])
    item_one_id = int(row['Item1'])
    item_two_id = int(row['Item2'])
    item_one_rank = final_ranking[user_id,item_one_id]
    item_two_rank = final_ranking[user_id,item_two_id]
    print("item_one_rank = %.10f ; item_two_rank = %.10f " % ( item_one_rank, item_two_rank))
    result = 0
    if item_two_rank < item_one_rank:result = 1
    mask = (random_test['UserID'] == user_id) & (random_test['Item1'] == item_one_id) & (random_test['Item2'] == item_two_id)
    random_test.loc[mask, 'bitClassification'] = result
random_test.to_csv (r'random_300339785_201314796_203016217.csv', index = None, header=True)

item_one_rank = 3517.0000000000 ; item_two_rank = 450.0000000000 
item_one_rank = 1030.0000000000 ; item_two_rank = 3176.0000000000 
item_one_rank = 1700.0000000000 ; item_two_rank = 373.0000000000 
item_one_rank = 385.0000000000 ; item_two_rank = 2839.0000000000 
item_one_rank = 1794.0000000000 ; item_two_rank = 422.0000000000 
item_one_rank = 3476.0000000000 ; item_two_rank = 1561.0000000000 
item_one_rank = 2096.0000000000 ; item_two_rank = 943.0000000000 
item_one_rank = 1153.0000000000 ; item_two_rank = 3457.0000000000 
item_one_rank = 1229.0000000000 ; item_two_rank = 235.0000000000 
item_one_rank = 1.0000000000 ; item_two_rank = 2594.0000000000 
item_one_rank = 702.0000000000 ; item_two_rank = 123.0000000000 
item_one_rank = 156.0000000000 ; item_two_rank = 2713.0000000000 
item_one_rank = 642.0000000000 ; item_two_rank = 1064.0000000000 
item_one_rank = 2787.0000000000 ; item_two_rank = 3594.0000000000 
item_one_rank = 944.0000000000 ; item_two_rank = 3569.0000000000 
item_one_

item_one_rank = 2685.0000000000 ; item_two_rank = 2020.0000000000 
item_one_rank = 870.0000000000 ; item_two_rank = 274.0000000000 
item_one_rank = 183.0000000000 ; item_two_rank = 448.0000000000 
item_one_rank = 2760.0000000000 ; item_two_rank = 1277.0000000000 
item_one_rank = 1358.0000000000 ; item_two_rank = 380.0000000000 
item_one_rank = 70.0000000000 ; item_two_rank = 1181.0000000000 
item_one_rank = 2115.0000000000 ; item_two_rank = 127.0000000000 
item_one_rank = 2068.0000000000 ; item_two_rank = 1911.0000000000 
item_one_rank = 1555.0000000000 ; item_two_rank = 314.0000000000 
item_one_rank = 31.0000000000 ; item_two_rank = 721.0000000000 
item_one_rank = 2010.0000000000 ; item_two_rank = 239.0000000000 
item_one_rank = 2959.0000000000 ; item_two_rank = 574.0000000000 
item_one_rank = 2103.0000000000 ; item_two_rank = 2327.0000000000 
item_one_rank = 3319.0000000000 ; item_two_rank = 794.0000000000 
item_one_rank = 2784.0000000000 ; item_two_rank = 99.0000000000 
item_one_ran

item_one_rank = 1798.0000000000 ; item_two_rank = 2320.0000000000 
item_one_rank = 259.0000000000 ; item_two_rank = 163.0000000000 
item_one_rank = 1973.0000000000 ; item_two_rank = 938.0000000000 
item_one_rank = 3463.0000000000 ; item_two_rank = 167.0000000000 
item_one_rank = 387.0000000000 ; item_two_rank = 3495.0000000000 
item_one_rank = 1537.0000000000 ; item_two_rank = 206.0000000000 
item_one_rank = 2461.0000000000 ; item_two_rank = 4.0000000000 
item_one_rank = 440.0000000000 ; item_two_rank = 2257.0000000000 
item_one_rank = 2778.0000000000 ; item_two_rank = 1042.0000000000 
item_one_rank = 1425.0000000000 ; item_two_rank = 1371.0000000000 
item_one_rank = 152.0000000000 ; item_two_rank = 3474.0000000000 
item_one_rank = 310.0000000000 ; item_two_rank = 1723.0000000000 
item_one_rank = 111.0000000000 ; item_two_rank = 2226.0000000000 
item_one_rank = 2566.0000000000 ; item_two_rank = 78.0000000000 
item_one_rank = 8.0000000000 ; item_two_rank = 512.0000000000 
item_one_rank 

item_one_rank = 978.0000000000 ; item_two_rank = 913.0000000000 
item_one_rank = 19.0000000000 ; item_two_rank = 2769.0000000000 
item_one_rank = 3533.0000000000 ; item_two_rank = 1179.0000000000 
item_one_rank = 1349.0000000000 ; item_two_rank = 1098.0000000000 
item_one_rank = 1931.0000000000 ; item_two_rank = 181.0000000000 
item_one_rank = 1148.0000000000 ; item_two_rank = 1915.0000000000 
item_one_rank = 3481.0000000000 ; item_two_rank = 252.0000000000 
item_one_rank = 1881.0000000000 ; item_two_rank = 2599.0000000000 
item_one_rank = 1247.0000000000 ; item_two_rank = 1815.0000000000 
item_one_rank = 201.0000000000 ; item_two_rank = 2760.0000000000 
item_one_rank = 34.0000000000 ; item_two_rank = 1025.0000000000 
item_one_rank = 59.0000000000 ; item_two_rank = 553.0000000000 
item_one_rank = 1448.0000000000 ; item_two_rank = 1014.0000000000 
item_one_rank = 650.0000000000 ; item_two_rank = 1074.0000000000 
item_one_rank = 2715.0000000000 ; item_two_rank = 1578.0000000000 
item_one

item_one_rank = 347.0000000000 ; item_two_rank = 2859.0000000000 
item_one_rank = 1314.0000000000 ; item_two_rank = 36.0000000000 
item_one_rank = 3230.0000000000 ; item_two_rank = 2459.0000000000 
item_one_rank = 1715.0000000000 ; item_two_rank = 1062.0000000000 
item_one_rank = 552.0000000000 ; item_two_rank = 2243.0000000000 
item_one_rank = 788.0000000000 ; item_two_rank = 1290.0000000000 
item_one_rank = 158.0000000000 ; item_two_rank = 1213.0000000000 
item_one_rank = 1996.0000000000 ; item_two_rank = 1163.0000000000 
item_one_rank = 1587.0000000000 ; item_two_rank = 2313.0000000000 
item_one_rank = 479.0000000000 ; item_two_rank = 1580.0000000000 
item_one_rank = 951.0000000000 ; item_two_rank = 1834.0000000000 
item_one_rank = 1361.0000000000 ; item_two_rank = 2575.0000000000 
item_one_rank = 2167.0000000000 ; item_two_rank = 1124.0000000000 
item_one_rank = 844.0000000000 ; item_two_rank = 2272.0000000000 
item_one_rank = 3465.0000000000 ; item_two_rank = 455.0000000000 
item_

...

item_one_rank = 3162.0000000000 ; item_two_rank = 171.0000000000 
item_one_rank = 2782.0000000000 ; item_two_rank = 3528.0000000000 
item_one_rank = 3535.0000000000 ; item_two_rank = 773.0000000000 
item_one_rank = 1638.0000000000 ; item_two_rank = 540.0000000000 
item_one_rank = 2106.0000000000 ; item_two_rank = 1608.0000000000 
item_one_rank = 148.0000000000 ; item_two_rank = 2709.0000000000 
item_one_rank = 2098.0000000000 ; item_two_rank = 469.0000000000 
item_one_rank = 1180.0000000000 ; item_two_rank = 765.0000000000 
item_one_rank = 781.0000000000 ; item_two_rank = 3294.0000000000 
item_one_rank = 223.0000000000 ; item_two_rank = 1428.0000000000 
item_one_rank = 2130.0000000000 ; item_two_rank = 1937.0000000000 
item_one_rank = 62.0000000000 ; item_two_rank = 3635.0000000000 
item_one_rank = 2762.0000000000 ; item_two_rank = 3.0000000000 
item_one_rank = 1860.0000000000 ; item_two_rank = 236.0000000000 
item_one_rank = 864.0000000000 ; item_two_rank = 1640.0000000000 
item_one_r

item_one_rank = 3459.0000000000 ; item_two_rank = 4.0000000000 
item_one_rank = 3198.0000000000 ; item_two_rank = 175.0000000000 
item_one_rank = 2538.0000000000 ; item_two_rank = 1594.0000000000 
item_one_rank = 2638.0000000000 ; item_two_rank = 31.0000000000 
item_one_rank = 698.0000000000 ; item_two_rank = 1741.0000000000 
item_one_rank = 46.0000000000 ; item_two_rank = 2904.0000000000 
item_one_rank = 2401.0000000000 ; item_two_rank = 472.0000000000 
item_one_rank = 1285.0000000000 ; item_two_rank = 3516.0000000000 
item_one_rank = 1160.0000000000 ; item_two_rank = 113.0000000000 
item_one_rank = 993.0000000000 ; item_two_rank = 858.0000000000 
item_one_rank = 2107.0000000000 ; item_two_rank = 1706.0000000000 
item_one_rank = 1455.0000000000 ; item_two_rank = 671.0000000000 
item_one_rank = 3659.0000000000 ; item_two_rank = 1310.0000000000 
item_one_rank = 1283.0000000000 ; item_two_rank = 336.0000000000 
item_one_rank = 1656.0000000000 ; item_two_rank = 2543.0000000000 
item_one_r

item_one_rank = 910.0000000000 ; item_two_rank = 1816.0000000000 
item_one_rank = 172.0000000000 ; item_two_rank = 2891.0000000000 
item_one_rank = 1104.0000000000 ; item_two_rank = 1903.0000000000 
item_one_rank = 556.0000000000 ; item_two_rank = 36.0000000000 
item_one_rank = 703.0000000000 ; item_two_rank = 2697.0000000000 
item_one_rank = 63.0000000000 ; item_two_rank = 892.0000000000 
item_one_rank = 34.0000000000 ; item_two_rank = 689.0000000000 
item_one_rank = 1808.0000000000 ; item_two_rank = 39.0000000000 
item_one_rank = 2138.0000000000 ; item_two_rank = 2372.0000000000 
item_one_rank = 36.0000000000 ; item_two_rank = 266.0000000000 
item_one_rank = 20.0000000000 ; item_two_rank = 3464.0000000000 
item_one_rank = 2249.0000000000 ; item_two_rank = 797.0000000000 
item_one_rank = 315.0000000000 ; item_two_rank = 128.0000000000 
item_one_rank = 598.0000000000 ; item_two_rank = 1786.0000000000 
item_one_rank = 1075.0000000000 ; item_two_rank = 181.0000000000 
item_one_rank = 551

item_one_rank = 161.0000000000 ; item_two_rank = 3291.0000000000 
item_one_rank = 120.0000000000 ; item_two_rank = 3019.0000000000 
item_one_rank = 162.0000000000 ; item_two_rank = 228.0000000000 
item_one_rank = 1394.0000000000 ; item_two_rank = 855.0000000000 
item_one_rank = 143.0000000000 ; item_two_rank = 3246.0000000000 
item_one_rank = 1296.0000000000 ; item_two_rank = 785.0000000000 
item_one_rank = 537.0000000000 ; item_two_rank = 2092.0000000000 
item_one_rank = 354.0000000000 ; item_two_rank = 272.0000000000 
item_one_rank = 1250.0000000000 ; item_two_rank = 2727.0000000000 
item_one_rank = 2988.0000000000 ; item_two_rank = 1324.0000000000 
item_one_rank = 549.0000000000 ; item_two_rank = 275.0000000000 
item_one_rank = 3319.0000000000 ; item_two_rank = 29.0000000000 
item_one_rank = 1077.0000000000 ; item_two_rank = 1431.0000000000 
item_one_rank = 1305.0000000000 ; item_two_rank = 231.0000000000 
item_one_rank = 842.0000000000 ; item_two_rank = 689.0000000000 
item_one_ran

item_one_rank = 144.0000000000 ; item_two_rank = 3128.0000000000 
item_one_rank = 2415.0000000000 ; item_two_rank = 359.0000000000 
item_one_rank = 871.0000000000 ; item_two_rank = 1210.0000000000 
item_one_rank = 3542.0000000000 ; item_two_rank = 686.0000000000 
item_one_rank = 567.0000000000 ; item_two_rank = 362.0000000000 
item_one_rank = 737.0000000000 ; item_two_rank = 903.0000000000 
item_one_rank = 2159.0000000000 ; item_two_rank = 1027.0000000000 
item_one_rank = 201.0000000000 ; item_two_rank = 1564.0000000000 
item_one_rank = 290.0000000000 ; item_two_rank = 2329.0000000000 
item_one_rank = 1627.0000000000 ; item_two_rank = 713.0000000000 
item_one_rank = 1655.0000000000 ; item_two_rank = 102.0000000000 
item_one_rank = 134.0000000000 ; item_two_rank = 2405.0000000000 
item_one_rank = 198.0000000000 ; item_two_rank = 642.0000000000 
item_one_rank = 2001.0000000000 ; item_two_rank = 24.0000000000 
item_one_rank = 3557.0000000000 ; item_two_rank = 2503.0000000000 
item_one_ran

In [21]:
# filling out the popularity test file accroding to the predicted ranking
popularity_test['bitClassification'] = None
for i, record in popularity_test.iterrows():
    row = record
    user_id = int(row['UserID'])
    item_one_id = int(row['Item1'])
    item_two_id = int(row['Item2'])
    item_one_rank = final_ranking[user_id,item_one_id]
    item_two_rank = final_ranking[user_id,item_two_id]
    print("item_one_rank = %.10f ; item_two_rank = %.10f " % ( item_one_rank, item_two_rank))
    result = 0
    if item_two_rank > item_one_rank:result = 1
    mask = (popularity_test['UserID'] == user_id) & (popularity_test['Item1'] == item_one_id) & (popularity_test['Item2'] == item_two_id)
    popularity_test.loc[mask, 'bitClassification'] = result
popularity_test.to_csv (r'popularity_300339785_201314796_203016217.csv', index = None, header=True)

item_one_rank = 511.0000000000 ; item_two_rank = 1499.0000000000 
item_one_rank = 2169.0000000000 ; item_two_rank = 1665.0000000000 
item_one_rank = 102.0000000000 ; item_two_rank = 32.0000000000 
item_one_rank = 190.0000000000 ; item_two_rank = 2068.0000000000 
item_one_rank = 4.0000000000 ; item_two_rank = 768.0000000000 
item_one_rank = 603.0000000000 ; item_two_rank = 709.0000000000 
item_one_rank = 1975.0000000000 ; item_two_rank = 554.0000000000 
item_one_rank = 811.0000000000 ; item_two_rank = 1807.0000000000 
item_one_rank = 762.0000000000 ; item_two_rank = 252.0000000000 
item_one_rank = 151.0000000000 ; item_two_rank = 1276.0000000000 
item_one_rank = 946.0000000000 ; item_two_rank = 1616.0000000000 
item_one_rank = 1473.0000000000 ; item_two_rank = 700.0000000000 
item_one_rank = 303.0000000000 ; item_two_rank = 35.0000000000 
item_one_rank = 1305.0000000000 ; item_two_rank = 886.0000000000 
item_one_rank = 1313.0000000000 ; item_two_rank = 197.0000000000 
item_one_rank = 23

item_one_rank = 716.0000000000 ; item_two_rank = 1228.0000000000 
item_one_rank = 2757.0000000000 ; item_two_rank = 1743.0000000000 
item_one_rank = 1634.0000000000 ; item_two_rank = 2245.0000000000 
item_one_rank = 432.0000000000 ; item_two_rank = 399.0000000000 
item_one_rank = 342.0000000000 ; item_two_rank = 962.0000000000 
item_one_rank = 11.0000000000 ; item_two_rank = 312.0000000000 
item_one_rank = 657.0000000000 ; item_two_rank = 956.0000000000 
item_one_rank = 1537.0000000000 ; item_two_rank = 2785.0000000000 
item_one_rank = 1486.0000000000 ; item_two_rank = 1798.0000000000 
item_one_rank = 2338.0000000000 ; item_two_rank = 1775.0000000000 
item_one_rank = 1163.0000000000 ; item_two_rank = 65.0000000000 
item_one_rank = 593.0000000000 ; item_two_rank = 37.0000000000 
item_one_rank = 4.0000000000 ; item_two_rank = 217.0000000000 
item_one_rank = 305.0000000000 ; item_two_rank = 67.0000000000 
item_one_rank = 834.0000000000 ; item_two_rank = 1474.0000000000 
item_one_rank = 85

item_one_rank = 2753.0000000000 ; item_two_rank = 165.0000000000 
item_one_rank = 571.0000000000 ; item_two_rank = 2497.0000000000 
item_one_rank = 1082.0000000000 ; item_two_rank = 1029.0000000000 
item_one_rank = 1193.0000000000 ; item_two_rank = 670.0000000000 
item_one_rank = 127.0000000000 ; item_two_rank = 582.0000000000 
item_one_rank = 2083.0000000000 ; item_two_rank = 500.0000000000 
item_one_rank = 1233.0000000000 ; item_two_rank = 205.0000000000 
item_one_rank = 2175.0000000000 ; item_two_rank = 555.0000000000 
item_one_rank = 818.0000000000 ; item_two_rank = 1678.0000000000 
item_one_rank = 643.0000000000 ; item_two_rank = 446.0000000000 
item_one_rank = 28.0000000000 ; item_two_rank = 540.0000000000 
item_one_rank = 3054.0000000000 ; item_two_rank = 2517.0000000000 
item_one_rank = 1761.0000000000 ; item_two_rank = 724.0000000000 
item_one_rank = 32.0000000000 ; item_two_rank = 36.0000000000 
item_one_rank = 776.0000000000 ; item_two_rank = 205.0000000000 
item_one_rank = 

item_one_rank = 1900.0000000000 ; item_two_rank = 106.0000000000 
item_one_rank = 22.0000000000 ; item_two_rank = 244.0000000000 
item_one_rank = 310.0000000000 ; item_two_rank = 143.0000000000 
item_one_rank = 1647.0000000000 ; item_two_rank = 1009.0000000000 
item_one_rank = 1437.0000000000 ; item_two_rank = 482.0000000000 
item_one_rank = 32.0000000000 ; item_two_rank = 2406.0000000000 
item_one_rank = 1239.0000000000 ; item_two_rank = 2193.0000000000 
item_one_rank = 166.0000000000 ; item_two_rank = 87.0000000000 
item_one_rank = 209.0000000000 ; item_two_rank = 620.0000000000 
item_one_rank = 405.0000000000 ; item_two_rank = 343.0000000000 
item_one_rank = 367.0000000000 ; item_two_rank = 141.0000000000 
item_one_rank = 107.0000000000 ; item_two_rank = 128.0000000000 
item_one_rank = 257.0000000000 ; item_two_rank = 651.0000000000 
item_one_rank = 1334.0000000000 ; item_two_rank = 2204.0000000000 
item_one_rank = 339.0000000000 ; item_two_rank = 812.0000000000 
item_one_rank = 950

item_one_rank = 1190.0000000000 ; item_two_rank = 927.0000000000 
item_one_rank = 5.0000000000 ; item_two_rank = 2178.0000000000 
item_one_rank = 604.0000000000 ; item_two_rank = 32.0000000000 
item_one_rank = 117.0000000000 ; item_two_rank = 321.0000000000 
item_one_rank = 1201.0000000000 ; item_two_rank = 3417.0000000000 
item_one_rank = 523.0000000000 ; item_two_rank = 854.0000000000 
item_one_rank = 2583.0000000000 ; item_two_rank = 1249.0000000000 
item_one_rank = 126.0000000000 ; item_two_rank = 1040.0000000000 
item_one_rank = 241.0000000000 ; item_two_rank = 99.0000000000 
item_one_rank = 1493.0000000000 ; item_two_rank = 2198.0000000000 
item_one_rank = 2982.0000000000 ; item_two_rank = 3499.0000000000 
item_one_rank = 74.0000000000 ; item_two_rank = 807.0000000000 
item_one_rank = 20.0000000000 ; item_two_rank = 49.0000000000 
item_one_rank = 321.0000000000 ; item_two_rank = 1227.0000000000 
item_one_rank = 821.0000000000 ; item_two_rank = 2547.0000000000 
item_one_rank = 119

...

item_one_rank = 516.0000000000 ; item_two_rank = 534.0000000000 
item_one_rank = 1184.0000000000 ; item_two_rank = 169.0000000000 
item_one_rank = 725.0000000000 ; item_two_rank = 85.0000000000 
item_one_rank = 307.0000000000 ; item_two_rank = 2537.0000000000 
item_one_rank = 32.0000000000 ; item_two_rank = 75.0000000000 
item_one_rank = 56.0000000000 ; item_two_rank = 2023.0000000000 
item_one_rank = 1379.0000000000 ; item_two_rank = 1039.0000000000 
item_one_rank = 539.0000000000 ; item_two_rank = 569.0000000000 
item_one_rank = 278.0000000000 ; item_two_rank = 1135.0000000000 
item_one_rank = 14.0000000000 ; item_two_rank = 988.0000000000 
item_one_rank = 280.0000000000 ; item_two_rank = 1956.0000000000 
item_one_rank = 358.0000000000 ; item_two_rank = 139.0000000000 
item_one_rank = 402.0000000000 ; item_two_rank = 256.0000000000 
item_one_rank = 770.0000000000 ; item_two_rank = 2586.0000000000 
item_one_rank = 601.0000000000 ; item_two_rank = 1663.0000000000 
item_one_rank = 346.0

item_one_rank = 172.0000000000 ; item_two_rank = 676.0000000000 
item_one_rank = 23.0000000000 ; item_two_rank = 1560.0000000000 
item_one_rank = 1695.0000000000 ; item_two_rank = 106.0000000000 
item_one_rank = 780.0000000000 ; item_two_rank = 1437.0000000000 
item_one_rank = 1750.0000000000 ; item_two_rank = 528.0000000000 
item_one_rank = 282.0000000000 ; item_two_rank = 851.0000000000 
item_one_rank = 776.0000000000 ; item_two_rank = 1993.0000000000 
item_one_rank = 319.0000000000 ; item_two_rank = 513.0000000000 
item_one_rank = 2922.0000000000 ; item_two_rank = 1330.0000000000 
item_one_rank = 11.0000000000 ; item_two_rank = 926.0000000000 
item_one_rank = 289.0000000000 ; item_two_rank = 2157.0000000000 
item_one_rank = 1433.0000000000 ; item_two_rank = 1099.0000000000 
item_one_rank = 1.0000000000 ; item_two_rank = 2220.0000000000 
item_one_rank = 2027.0000000000 ; item_two_rank = 433.0000000000 
item_one_rank = 49.0000000000 ; item_two_rank = 1733.0000000000 
item_one_rank = 1

item_one_rank = 393.0000000000 ; item_two_rank = 76.0000000000 
item_one_rank = 576.0000000000 ; item_two_rank = 1662.0000000000 
item_one_rank = 294.0000000000 ; item_two_rank = 273.0000000000 
item_one_rank = 942.0000000000 ; item_two_rank = 2.0000000000 
item_one_rank = 169.0000000000 ; item_two_rank = 55.0000000000 
item_one_rank = 396.0000000000 ; item_two_rank = 415.0000000000 
item_one_rank = 470.0000000000 ; item_two_rank = 430.0000000000 
item_one_rank = 1106.0000000000 ; item_two_rank = 56.0000000000 
item_one_rank = 912.0000000000 ; item_two_rank = 333.0000000000 
item_one_rank = 3346.0000000000 ; item_two_rank = 37.0000000000 
item_one_rank = 990.0000000000 ; item_two_rank = 266.0000000000 
item_one_rank = 1409.0000000000 ; item_two_rank = 291.0000000000 
item_one_rank = 3.0000000000 ; item_two_rank = 1152.0000000000 
item_one_rank = 244.0000000000 ; item_two_rank = 238.0000000000 
item_one_rank = 671.0000000000 ; item_two_rank = 3413.0000000000 
item_one_rank = 1733.000000

item_one_rank = 1240.0000000000 ; item_two_rank = 3532.0000000000 
item_one_rank = 1267.0000000000 ; item_two_rank = 1428.0000000000 
item_one_rank = 1249.0000000000 ; item_two_rank = 196.0000000000 
item_one_rank = 85.0000000000 ; item_two_rank = 1231.0000000000 
item_one_rank = 68.0000000000 ; item_two_rank = 203.0000000000 
item_one_rank = 54.0000000000 ; item_two_rank = 541.0000000000 
item_one_rank = 297.0000000000 ; item_two_rank = 71.0000000000 
item_one_rank = 128.0000000000 ; item_two_rank = 19.0000000000 
item_one_rank = 246.0000000000 ; item_two_rank = 491.0000000000 
item_one_rank = 194.0000000000 ; item_two_rank = 198.0000000000 
item_one_rank = 4.0000000000 ; item_two_rank = 938.0000000000 
item_one_rank = 1761.0000000000 ; item_two_rank = 1784.0000000000 
item_one_rank = 259.0000000000 ; item_two_rank = 1237.0000000000 
item_one_rank = 860.0000000000 ; item_two_rank = 700.0000000000 
item_one_rank = 1033.0000000000 ; item_two_rank = 369.0000000000 
item_one_rank = 752.00

item_one_rank = 800.0000000000 ; item_two_rank = 123.0000000000 
item_one_rank = 3430.0000000000 ; item_two_rank = 989.0000000000 
item_one_rank = 213.0000000000 ; item_two_rank = 107.0000000000 
item_one_rank = 1930.0000000000 ; item_two_rank = 115.0000000000 
item_one_rank = 1064.0000000000 ; item_two_rank = 20.0000000000 
item_one_rank = 2943.0000000000 ; item_two_rank = 36.0000000000 
item_one_rank = 2412.0000000000 ; item_two_rank = 2557.0000000000 
item_one_rank = 76.0000000000 ; item_two_rank = 726.0000000000 
item_one_rank = 1097.0000000000 ; item_two_rank = 634.0000000000 
item_one_rank = 1964.0000000000 ; item_two_rank = 331.0000000000 
item_one_rank = 757.0000000000 ; item_two_rank = 450.0000000000 
item_one_rank = 825.0000000000 ; item_two_rank = 1017.0000000000 
item_one_rank = 246.0000000000 ; item_two_rank = 88.0000000000 
item_one_rank = 185.0000000000 ; item_two_rank = 239.0000000000 
item_one_rank = 1438.0000000000 ; item_two_rank = 13.0000000000 
item_one_rank = 2184